In [1]:
import pandas as pd
import numpy as np
import json, pickle, os
import urllib2


In [2]:
def get_ids(json_file):
    # takes a json file with list of articles meta data including a field named 'istex_id'
    # retuns numpy array with the istex_id of the articles
    ids = np.arange(len(json_file), dtype=np.object)
    for (i,s) in enumerate(json_file):
        ids[i] = s['istex_id']
    return ids

In [52]:
def get_abstract(istex_id):
    response = urllib2.urlopen('https://api.istex.fr/document/'+istex_id)
    data = json.load(response, encoding=None)
    try:
        abst = data['abstract']
        return abst
    except:
        print istex_id
        return data['title']

In [4]:
seed = json.load(open('../RecSys_Exp_files/ISTEX_MentalRotation/sample_data/sportArticlesAsIstex_UniqID_182.json','r'))
test = json.load(open('../RecSys_Exp_files/ISTEX_MentalRotation/sample_data/MentalRotationInMetaDataIstexWithoutAnnotated.json','r'))
seed_ids = get_ids(seed)
test_ids = get_ids(test)

In [17]:
seed_abstracts = np.array(range(len(seed_ids)), dtype=np.object)
for i in range(len(seed)):
    seed_abstracts[i] = seed[i]['abstract'].split('.')

In [37]:
def get_sentances(abstracts):
#create the list of sentances exluding empty or very short sentances
    sentances = []
    for abstract in abstracts:
        for sentance in abstract:
            if len(sentance) > 21:
                sentances.append(sentance)
    return sentances
seed_sentances = get_sentances(seed_abstracts)
len(seed_sentances)

1310

In [15]:
# a seed article without an abstract
seed[151]

{u'KeyWords': [u'orientation-dependant', u'shape recognition'],
 u'abstract': u'',
 u'authors': [{u'name': u'Michael J. Tarr'}, {u'name': u'Steven Pinker'}],
 u'doi': u'10.1111/j.1467-9280.1991.tb00135.x',
 u'host': {u'corpusName': u' Association for Psychological Science',
  u'title': u'Psychological Science',
  u'volume': u'2'},
 u'istex_id': u'ANNOTATED001MENTAL000ROTATION10100000136',
 u'publicationDate': u'1991',
 u'source': [u'UCBL'],
 u'title': u'Orientation-Dependent Mechanisms in Shape Recognition: Further Issues'}

In [47]:
def get_abstracts(ids):
    abstracts = np.array(range(len(ids)), dtype=np.object)
    for (i, t_id) in enumerate(ids):
        abstracts[i] = get_abstract(t_id).split('.')
    return abstracts
test_abstracts = get_abstracts(test_ids)

In [48]:
test_sentances = get_sentances(test_abstracts)
len(test_sentances)

1551

In [55]:
def bring_results_sentances(file_path, id_column):
    df = pd.DataFrame()
    top200 = np.array(df.from_csv(file_path)[id_column])
    abstracts = get_abstracts(top200)
    sentances = get_sentances(abstracts)
    return sentances

In [77]:
#bring in the active learning top 200 results
active_sentances = bring_results_sentances('top200_activelearn.csv', '0')
len(active_sentances)

51306323120126D81D0E88A1A6A69BABC1708F9D
823F414AA910D7417970A91A4840656D09B1ADE1
D0EA2A2E77A4034C3CB2158627D7221427AC7814
0EBBF1E1DC67BF2DEE7A8A040C5BE9B2F458B3D7
93908E1D3B68509A775E449D755F6E7615FDD6E8
DE0049F2163279B6797EDEB287C0A484B5EA5CE4
97197631FC7844EAB0F5DE25112218FA63375173
E72E444D4F707D3E7495F0FA554CF8829E206C46
E6E5207357437DFDCA56FDB1755E13406C1E0616
0DC64D97700639752B7416A04BA6AAE7EACB5F1B
93C4C8EF5ED606BC28823674848D9C9AC09633D6
92A27CBC644960A8F6084C1E955E055B2CAD7D13
00EE087C6E790885C9036E261B7D06097D96C79A


1414

In [76]:
def get_top_k(k, file_path, article_column):
    df = pd.DataFrame()
    top = np.array(range(k), dtype=np.object)
    for (i, article) in enumerate(np.array(df.from_csv(file_path)[article_column])):
        if i > k-1:
            break
        top[i] = article.split('/')[4]
    return top.tolist()

In [73]:
def bring_results_sentances_from_list(top200):
    abstracts = get_abstracts(top200)
    sentances = get_sentances(abstracts)
    return sentances

In [74]:
#bring in 3SB top 200 results
qbe_top200 = get_top_k(200, 'QbE.csv', 'Articles')
qbe_sentance =  bring_results_sentances_from_list(qbe_top200)
len(qbe_sentance)

9A1BBD3AC408F42F669B2F2755730FB5DCF2AFC8
34FA03814FB1B8C5DB7E1124AFB6933263ED9406
0987992DAB579C83521BABD54F3DD7840B5709F0


1571

In [75]:
#bring in MLT top 200 results
mlt_top200 = get_top_k(200, 'MLT.csv', 'Article')
mlt_sentance =  bring_results_sentances_from_list(mlt_top200)
len(mlt_sentance)

8A14A0D5DEEA9D91EC6C8B22064490EFDFB3506F
9D962699AE087CE8DE5B2D7B61ECBEE5C40A6F04
123D68EFDE046B70CC1AD52B1E9951246E1C3D27
27468151CE649DAA5A6792D4D7C6D968AFD26E6E
E93468E68B514EA825BD250FC5F42587BF4A79E6
0706EC3B1C7B1501212E58260E702214E38A1ED1
0917DB247E3573222DF2F307C4E59D94CC135ACC
14012FAE6879AB6E9F3BB4FF2BBF6BC7E4723EC5
94C991E0C141B84FFB78A5CE1453FF253AFBD7B5
1A8EFA52B147739BE8FC7E01483BDBA000BE442B
E1DACA0569047B9DFFCB3DA2A82A89A03726B9EB
6406A034CC2894C42B8861B24A36AB32652987CE
740F6FA421B882270AE202D1C5F7551A0C3FCF6F
35966DD015D8F3B393BEF2BFDADFCC79F939B1EA
ED37CECF0769C38B7319121EA5EC4D21F60F17E4
2E714DD20F5AAEC95139FE946F2FF0FE406DB568
757B3BA30A8182AE01E40E8F0EC8376015217535
65DA0042682AE20F37987B05FC8397F03F3C4AE1
E1755C76C3945AD4BC27F8F98C9E9E21494186F9
6CF2FEA432D1261AEAF86ABA4B01D7DC71F44C69
8BC7E0732DB69219F7EF74243E2B788EF321EA20
A4BB24D04156565D0987C5E150A6C767CCD2D845
CD889F8E46180B12558B3A33C7C742BC6C8955E0
10C0252C82B1AC3C4A45129CC592853615833E49
496E07332CB8693C

1516

In [85]:
def auto_pairs_gen(result_sents, outfile):
    sent_pairs = []
    for s1 in result_sents:
        for s2 in seed_sentances:
            sent_pairs.append((s1, s2, 0.0))
    df = pd.DataFrame(sent_pairs, columns=['Sent1', 'Sent2', 'Score'])
    df.to_csv('sent_sem_exp_'+outfile, sep='~', index=None, encoding='utf-8')
    return 'retuned file '+outfile


In [83]:
auto_pairs_gen(active_sentances, 'active_pairs')

'retuned file active_pairs'

In [86]:
auto_pairs_gen(qbe_sentance, 'qbe_pairs')
auto_pairs_gen(mlt_sentance, 'mlt_pairs')

'retuned file mlt_pairs'

In [32]:
seed_sents = pd.read_csv(open('sample_data/3seeds.txt'), header=None, sep='\n')
active1_sents = pd.read_csv(open('sample_data/active1.txt'), header=None, sep='\n')
active2_sents = pd.read_csv(open('sample_data/active2.txt'), header=None, sep='\n')
active3_sents = pd.read_csv(open('sample_data/active3.txt'), header=None, sep='\n')
phase1_1_sents = pd.read_csv(open('sample_data/phase1_1.txt'), header=None, sep='\n')
phase1_2_sents = pd.read_csv(open('sample_data/phase1_2.txt'), header=None, sep='\n')
phase1_3_sents = pd.read_csv(open('sample_data/phase1_3.txt'), header=None, sep='\n')

In [84]:
def pairs_gen(active_sents, outfile):
    sent_pairs = []
    for s1 in active_sents[0]:
        for s2 in seed_sents[0]:
            sent_pairs.append((s1, s2, 0.0))
    df = pd.DataFrame(sent_pairs, columns=['Sent1', 'Sent2', 'Score'])
    df.to_csv('sent_sem_exp_'+outfile, sep='~', index=None)
    return 'retuned file '+outfile


In [35]:
pairs_gen(active1_sents, 'active1_pairs.csv')
pairs_gen(active2_sents, 'active2_pairs.csv')
pairs_gen(active3_sents, 'active3_pairs.csv')
pairs_gen(phase1_1_sents, 'phase1_1_pairs.csv')
pairs_gen(phase1_2_sents, 'phase1_2_pairs.csv')
pairs_gen(phase1_3_sents, 'phase1_3_pairs.csv')

'retuned file phase1_3_pairs.csv'

In [12]:
active1_pairs = pd.read_csv(open('sent_sem_exp/score_active1_pairs.csv'), header=0, sep='\t')
active1_pairs

,Sent1,Sent2,Score
0,Generation of skeletal and multipart mental vi...,Effects of Using Mental and Manual Rotation Tr...,1.920951
1,Generation of skeletal and multipart mental vi...,The goal of this project was to examine how tr...,2.942941
2,Generation of skeletal and multipart mental vi...,"In Experiment 1, we examined improvement on a ...",1.764000
3,Generation of skeletal and multipart mental vi...,Practice in mental but not manual rotation lea...,0.956246
4,Generation of skeletal and multipart mental vi...,"In Experiment 2, we examined improvement on a ...",0.718133
5,Generation of skeletal and multipart mental vi...,"In this experiment, both mental and manual rot...",1.541414
6,Generation of skeletal and multipart mental vi...,These results suggest common processes in ment...,2.190217
7,Generation of skeletal and multipart mental vi...,The functional role of dorso-lateral premotor ...,2.064954
8,Generation of skeletal and multipart mental vi...,We examined whether body parts attached to abs...,0.773612
9,Generation of skeletal and multipart mental vi...,"In Experiment 1, standard cube combinations re...",1.140523
